In [16]:
import os
from dotenv import load_dotenv
import cv2
import numpy as np
import json

load_dotenv()

True

In [17]:
CURRENT_PATH = os.getenv('CURRENT_PATH')
with open(CURRENT_PATH + "/factory/context/community_mgmt/materials.json", "r") as f:
        materials = json.load(f)
        print(materials["top_text"])

[{'emotion': 'angry', 'situation': 'missed_deadline', 'text': 'Why do Devs keep missing deadline!?!'}, {'emotion': 'frustration', 'situation': 'missed_deadline', 'text': 'Devs keep missing deadline?'}, {'emotion': 'frustration', 'situation': 'community_hopium', 'text': 'Smelling Hopium In the Air?'}, {'emotion': 'frustration', 'situation': 'productivity_encouragement', 'text': 'OG Team too late?'}, {'emotion': 'Happiness', 'situation': 'new_app_launch', 'text': 'Make sure it actually works?'}, {'emotion': 'Desperation', 'situation': 'original_team_MIA', 'text': "Te isn't showing up?"}]


In [18]:
from factory import *
from factory_widgets import *

In [19]:
## Backend Interface is 
# 1) Grab emotion and situation
# 2) Search through materials
# 3) Create appropriate combination


In [48]:
emotion, situation = "Happiness", "new_app_launch"

pic = emotion_and_situation_to_picture(emotion, situation, materials)
tt = emotion_and_situation_to_top_text(emotion, situation, materials)
bt = emotion_and_situation_to_bottom_text(emotion, situation, materials)

In [55]:
pic_path = None
for item in materials["pic_path"]:
    if item["emotion"] == emotion and item["situation"] == situation:
        pic_path = item["text"]

In [49]:
tt, bt

('Make sure it actually works?', 'App finally launched.')

In [50]:
h, w, d = pic.shape
h, w, d

(512, 512, 3)

In [51]:
color = grab_background_color(pic, "corners_and_edges")
color

(243, 138, 84)

In [24]:
r, g, b = color
font_color = (int(255 - r), int(255 - g), int(255 - b))
font_color

(255, 255, 255)

In [25]:
for i,j in zip(color, font_color):
    print(type(i), type(j))

<class 'numpy.uint8'> <class 'int'>
<class 'numpy.uint8'> <class 'int'>
<class 'numpy.uint8'> <class 'int'>


In [26]:
top_pad = create_pad_and_text_wif_font_scaling(
    color, h//3, w, d, 
    tt, (w//2, h//6), "center",
    font=cv2.FONT_HERSHEY_COMPLEX
)
bottom_pad = create_pad_and_text_wif_font_scaling(
    color, h//3, w, d, 
    bt, (w//2, h//6), "center",
    font=cv2.FONT_HERSHEY_COMPLEX
)
# bottom_pad = create_pad_and_text(
#     color, h//3, w, d, 
#     bt, (w//2, h//6), "center",
#     font=cv2.FONT_HERSHEY_COMPLEX, font_scale=0.55, font_color=font_color, font_thickness=1
# )



In [57]:
def change_background_color(image, new_background_color):
    mask = np.zeros(image.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    
    height, width = image.shape[:2]
    rect = (10, 10, width-10, height-10)
    
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    foreground = image * mask2[:, :, np.newaxis]

    background = np.full(image.shape, new_background_color, dtype=np.uint8)
    
    # Combine the foreground with the new background
    inverse_mask = 1 - mask2
    new_background = background * inverse_mask[:, :, np.newaxis]
    result = cv2.add(foreground, new_background)
    
    return result

In [69]:
!pip3 install keras


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [72]:
import cv2
import numpy as np

from segmentation_models import Unet
from segmentation_models import backbones

def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def preprocess_image(image, backbone_name='resnet34'):
    preprocessing_fn = backbones.get_preprocessing(backbone_name)
    image = cv2.resize(image, (256, 256))
    image = preprocessing_fn(image)
    return image

def segment_image(image, model):
    preprocessed_image = preprocess_image(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
    mask = model.predict(preprocessed_image)[0]
    mask = (mask > 0.5).astype(np.uint8)
    return mask

def change_background(image, mask, new_background_color=(0, 255, 0)):
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
    foreground = image * np.expand_dims(mask, axis=-1)
    background = np.full_like(image, new_background_color)
    background = background * np.expand_dims(1 - mask, axis=-1)
    result = foreground + background
    return result

# Load a pre-trained U-Net model
model = Unet('resnet34', encoder_weights='imagenet', input_shape=(256, 256, 3))

AttributeError: module 'keras.api.utils' has no attribute 'generic_utils'

In [61]:
new = change_background_color(pic, (94, 218, 250))
cv2.imwrite("background_change_test.png", new)

True

In [ ]:
## test neural net based background sourcing
# Usage example
mask = segment_image(pic, model)
result = change_background(image, mask, new_background_color=(94, 218, 250))
cv2.imwrite("background_change_neuralnet_test.png", new)

In [60]:
cv2.imwrite("original.png", pic)

True

In [31]:
new_pic = concatenate_images_inline([top_pad, pic, bottom_pad], horizontal=False)
cv2.imwrite("test_propaganda.png", new_pic)

True

In [53]:
npbc = change_background_color(pic, (94, 218, 250)) # bgr
cv2.imwrite("test_background_color_change.png", npbc)

True

In [80]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.0 MB/s eta 0:00:00a 0:00:010m

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [94]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)

ValueError: Pipeline cannot infer suitable model classes from briaai/RMBG-1.4

In [90]:
pic_path

'/Users/keonshikkim/Desktop/ART PROJECT/Materials/moncat/monc_cool.webp'